In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/krishnaik06/FSDSRegression/main/notebooks/data/gemstone.csv')

In [3]:
df=df.sample(n=10000)

Model Training:

In [4]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
177203,177203,1.01,Premium,F,VS2,61.1,58.0,6.47,6.44,3.95,6174
24579,24579,0.31,Ideal,G,VVS1,61.4,55.0,4.36,4.33,2.67,1046
66535,66535,1.08,Ideal,F,SI1,62.3,56.0,6.53,6.57,4.09,5449
153658,153658,0.30,Premium,G,SI1,61.6,60.0,4.30,4.27,2.64,595
59420,59420,0.41,Ideal,F,VVS2,61.8,57.0,4.78,4.75,2.95,1131


In [5]:
df=df.drop(labels=['id'],axis=1)

In [6]:
## Independent and Dependent feature
X= df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [7]:
X

,carat,cut,color,clarity,depth,table,x,y,z
177203,1.01,Premium,F,VS2,61.1,58.0,6.47,6.44,3.95
24579,0.31,Ideal,G,VVS1,61.4,55.0,4.36,4.33,2.67
66535,1.08,Ideal,F,SI1,62.3,56.0,6.53,6.57,4.09
153658,0.30,Premium,G,SI1,61.6,60.0,4.30,4.27,2.64
59420,0.41,Ideal,F,VVS2,61.8,57.0,4.78,4.75,2.95
...,...,...,...,...,...,...,...,...,...
87561,0.70,Ideal,I,VS1,62.1,56.0,5.69,5.71,3.54
36955,1.20,Premium,H,SI1,62.0,59.0,6.85,6.78,4.23
167809,0.51,Premium,G,VS1,61.8,60.0,5.15,5.11,3.17
69487,1.02,Ideal,G,SI1,62.3,56.0,6.38,6.44,3.99


In [8]:
Y

,price
177203,6174
24579,1046
66535,5449
153658,595
59420,1131
...,...
87561,2381
36955,6290
167809,1781
69487,4666


In [9]:
# Define which column should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [10]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [11]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [12]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']


In [13]:
from sklearn.impute import SimpleImputer ## Handling missing values
from sklearn.preprocessing import StandardScaler ## Handling Feature scaling
from sklearn.preprocessing import OrdinalEncoder ## ordinal Encoding 
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [14]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [15]:
## Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [16]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test= pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [17]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.999249,0.902593,-1.163724,-1.161575,-1.145939,-1.069552,-1.125682,-0.944226,-0.669881
1,-0.563692,0.622239,-0.631489,-0.464433,-0.508331,-0.426816,0.874515,-0.944226,0.001535
2,0.263867,1.182946,-0.631489,0.422838,0.375211,0.537287,-1.125682,-0.944226,-0.669881
3,1.592316,-0.125372,0.965216,1.482131,1.532014,1.501391,0.874515,-0.324597,0.001535
4,0.242089,0.341885,-0.631489,0.413784,0.393429,0.435034,0.874515,-1.563855,-1.341298


In [18]:
# Model Training
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [19]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [20]:
regression.coef_

array([[ 6469.00173698,  -176.71171962,   -59.23177136, -2302.76988416,
         -341.5610412 ,   282.04642538,    67.20930835,  -437.51605058,
          622.8719161 ]])

In [21]:
regression.intercept_

array([3959.28428571])

In [22]:
import numpy as num_pipeline
def evaluate_model(true, predicted):
    mae=mean_absolute_error(true, predicted)
    mse=mean_squared_error(true, predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true, predicted)
    return mae, rmse, r2_square

In [25]:
## train multiple models 
import numpy as np

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train, y_train)

    # Make predictions
    y_pred=model.predict(X_test)
    
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performave')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')

LinearRegression
Model Training Performave
RMSE: 1026.0823511478582
MAE: 673.645963373576
R2 score 93.50925264837961


Lasso
Model Training Performave
RMSE: 1026.1127819095586
MAE: 674.9369065387043
R2 score 93.50886764747864


Ridge
Model Training Performave
RMSE: 1026.2793792775028
MAE: 674.2158468307687
R2 score 93.50675970501604


ElasticNet
Model Training Performave
RMSE: 1561.2860616778216
MAE: 1060.4927530832804
R2 score 84.97220595154212


